<h2>Neural Network from scratch in Python</h2>
<h3>Make your own machine learning library</h3>
<p>MNIST FC --- version: notebook + lib</p>
<p>
Este notebook foi preparado para fazer a correção do trabalho 2 do cursos INF1310/INF2064 de 2021.2 da PUC-Rio.</p>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
import sklearn.metrics as mt
import time

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

<h2>Importa a rede implementada por cada alun@ </h2>
<p>
Cada alun@ deve desenvolver uma biblioteca que implemnta um rede MultiLayerPerceptron num arquivo 
Pyhton chamado "MLPSeuNome.py"  
que tenha a API que atenda este notebook.</br>
</p>

In [ ]:
from MLP_DavidJentjens import *
autor = "David Maisonnette Jentjens"

<h2>Carrega MNIST da rede</h2>

In [ ]:
#MNIST Dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

In [ ]:
fig, ax = plt.subplots(1,5, figsize=(18, 8))
for i in range(5):
  ax[i].imshow(x_train[i], cmap='gray')
  ax[i].set_title('Label ' + str(y_train[i]))
plt.show()

In [ ]:
# training data : 60000 samples
# reshape and normalize input data
x_train = x_train.reshape(x_train.shape[0], 32, 32, 3)
x_train = x_train.astype('float32')
x_train /= 255
# encode output which is a number in range [0,9] into a vector of size 10
# e.g. number 3 will become [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]

#y_train = np_utils.to_categorical(y_train)

# same for test data : 10000 samples
x_test = x_test.reshape(x_test.shape[0], 32, 32, 3)
x_test = x_test.astype('float32')
x_test /= 255
#y_test = np_utils.to_categorical(y_test)

print(x_train.shape)
print(y_train.shape)

<h2>Verifica o balanceamento das amostras</h2>

In [ ]:
#Count occurrences of each class in each set
values, counts_train = np.unique(y_train, return_counts=True)
values, counts_test = np.unique(y_test, return_counts=True)
print(f'Valores Y: {values}')
print(f'Histograma treinamento: {counts_train}')
print(f'Histograma test: {counts_test}')
print(f'Totais: treinamento={np.sum(counts_train)}, test={np.sum(counts_test)}')

<h2>Definição da rede</h2>

In [ ]:
np.random.seed(20)

net = Network()
net.layer(ConvLayer((32, 32, 3), (7, 7), 3))                
net.layer(ActLayer(sigmoid, sigmoid_prime))
net.layer(FlattenLayer())
net.layer(FCLayer(26*26*3, 100))
net.layer(ActLayer(sigmoid, sigmoid_prime))
net.layer(FCLayer(100, 50))
net.layer(ActLayer(sigmoid, sigmoid_prime))
net.layer(FCLayer(50, 10))  
net.layer(ActLayer(sigmoid, sigmoid_prime))
net.use(mse, mse_prime)

<h2>Treinamento</h2>

In [ ]:
from MLP_DavidJentjens import *

start = time.perf_counter()

err_train=net.fit(x_train[0:5000], y_train[0:5000], epochs=10, mini_batch=1, learning_rate=0.1)

stop=time.perf_counter()
time_train = stop-start
print(f'tempo de treinamento = {time_train:.2f} s')

In [ ]:
x = np.arange(0,len(err_train),1)
plt.figure(figsize=(8,5))
plt.plot(x,err_train)
plt.ylabel('erro')
plt.xlabel('epocas')
plt.show()

<h2>Teste</h2>

In [ ]:
start=time.perf_counter()
y_pred_one_hot = net.predict(x_test)
y_pred = np.argmax(y_pred_one_hot, axis=-1)
print(y_pred_one_hot.shape, y_pred.shape)
stop=time.perf_counter()
time_pred = stop-start
print(f'tempo de predicao = {time_pred:.3f} s')

In [ ]:
def plot_confusion_matrix(title,cm,class_names,size):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.
    
    Args:
       cm (array, shape = [n, n]): a confusion matrix of integer classes
       class_names (array, shape = [n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(size,size))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(title)
   
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names)
    plt.yticks(tick_marks, class_names)
    
    # Normalize the confusion matrix.
    #cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)
    
    # Use white text if squares are dark; otherwise black.
    threshold = 0.7*cm.max()
    
    #for i, j in itt.product(range(cm.shape[0]), range(cm.shape[1])):
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)
        
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

In [ ]:
# Calculate the confusion matrix using sklearn.metrics
CM = mt.confusion_matrix(y_test, y_pred)
fig=plot_confusion_matrix("Matriz de Confusao", CM, ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 6)

<h2>Medidas de erro</h2>

In [ ]:
num = den = 0
for i in range(10):
    for j in range(10):
        if i==j:
            num+=CM[i,j]
        den+=CM[i,j]
acuracia = 100*num/den
print(f'Acuracia={acuracia:.0f}%')

<h2>Salva os resultados</h2>

In [ ]:
fp = open("t4.csv","a")
fp.write(f'{autor};{time_train};{time_pred};{acuracia}\n')
fp.close()